1. Dengan data iris (dalam bentuk csv, data sudah disediakan), tuliskan kode dalam Python, agar data bisa digunakan dalam setting 5 cross-validation. 
2. Untuk membaca file csv bisa menggunakan library pandas atau csv reader dari Python.
3. Cross validation harus menggunakan native Python, tidak boleh menggunakan library. (Note:  kelas dalam tiap cv harus mendekati balance.)
4. Submission berupa Python notebook. Jangan lupa untuk me-run hasil untuk setiap blok kode 

## **1. Import library & Dataset**

In [ ]:
import numpy as np
import pandas as pd

from random import seed
from random import randrange

#Membaca dataset
df = pd.read_csv('/content/iris.csv')

#Menambahkan Class untuk setiap species
df['class'] = df['species']
df['class'] = df['class'].map({'setosa': 0, 'versicolor': 1, 'virginica': 2})
df["class"].unique()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
 5   class         150 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


## **2. Logistic Function**

<span id="5"></span>Untuk mendapatkan hasil antara 0 dan 1, sebuah fungsi, yang disebut **sigmoid**, digunakan untuk mengubah fungsi hipotesis kita. Ini didefinisikan sebagai
$$ $$
$$h_{\theta}(x) = g(\theta^{T} x)$$ 
$$ $$
dimana $h_{\theta}(x)$ adalah fungsi hipotesis, $x$ adalah catatan tunggal dan 
$$ $$
$$g(z)=\dfrac{1}{1+e^{-z}}$$
$$ $$
Dengan menggunakan $g(\theta^{T} x)$, kita memperoleh probabilitas dan jika $h_{\theta}(x) \geq 0.5$, kita mendapatkan $y=1$; jika $h_{\theta}(x) < 0.5$, kita mendapatkan $y=0$. Selanjutnya, ketika $z \geq 0$, $g(z) \geq 0.5$ adalah detail lainnya. Jadi, jika $\theta^{T} x \geq 0$, maka $y=1$.

In [ ]:
# Fungsi Sigmoid 
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

Kita tidak dapat menggunakan fungsi biaya yang sama dengan yang kami gunakan untuk regresi linier karena fungsi logistik akan menyebabkan output menjadi bergelombang, menyebabkan banyak local optima. Dengan kata lain, itu bukan fungsi cembung. Itu sebabnya kita perlu mendefinisikan fungsi cost yang berbeda untuk regresi logistik. Ini secara sederhana didefinisikan sebagai
$$ $$
$$J(\theta) = \dfrac{1}{m} \sum^{m}_{i=1}Cost(h_{\theta}(x^{(i)}), y^{(i )})$$
$$ $$
di mana
$$ $$
$$Cost(h_{\theta}(x^{(i)}), y^{(i)})=-y^{(i)} \; log(h_{\theta}(x^{(i)}))-(1-y^{(i)}) \; log(1-j_{\theta}(x^{(i)}))$$
$$ $$
Sebagai pemeriksaan kewarasan, $J(\theta)$ dapat diplot atau dicetak sebagai fungsi dari jumlah iterasi untuk memastikan bahwa $J(\theta)$ **berkurang pada setiap iterasi**, yang menunjukkan bahwa itu konvergen dengan benar. Pada titik ini, pilihan $\alpha$ penting. Jika kita memilih nilai $\alpha$ yang tinggi atau kecil, kita mungkin mengalami masalah tentang konvergensi.<span id="6"></span>

In [ ]:
# Fungsi Cost
def J(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

Untuk menemukan nilai $\theta$ yang meminimalkan fungsi cost, saya menggunakan penurunan gradien yang ditunjukkan secara matematis sebagai
$$ $$
$$\textbf{Repeat}\{ \; \theta_{j}:= \theta_{j}-\alpha \dfrac{\partial}{\partial \theta_{j}}J(\theta) \; \} \; di mana \; j \in \{0,1,2,...,n \}$$
$$ $$
$$atau$$
$$ $$
$$\textbf{Repeat}\{ \; \theta_{j}:= \theta_{j}-\dfrac{\alpha}{m} \sum^{m}_{i=1} (h_{\theta}(x^{(i)})- y^{(i)}) \; x_{j}^{(i)} \; \} \; di mana \; j \in \{0,1,2,...,n \}$$
$$ $$
Algoritma terlihat identik dengan regresi linier tetapi perlu diketahui bahwa kali ini fungsi $h_{\theta}(x^{(i)})$ memiliki **definisi berbeda** dan itulah sebabnya, keduanya tidak sama.

Saya juga ingin menjelaskan **regularisasi**. Regularisasi dirancang untuk mengatasi masalah overfitting dan undefitting. Untuk memulai dengan **overfitting**, itu berarti varians tinggi dan biasanya disebabkan oleh fungsi rumit yang menciptakan banyak kurva dan sudut yang tidak perlu yang tidak terkait dengan data. Fungsi ini cocok untuk melatih data dengan baik tetapi dapat menyebabkan hasil yang buruk untuk set pengujian. Di sisi lain, **underfitting** berarti varians rendah dan model yang sangat sederhana. Ini juga dapat menyebabkan hasil yang buruk. Dalam situasi ini, kita perlu menyesuaikan fitur secara manual atau menggunakan beberapa algoritma pemilihan model yang membawa beban kerja ekstra. Sebaliknya, ketika kita menerapkan regularisasi, semua fitur disimpan dan model menyesuaikan $\theta_{j}$. Ini terutama berfungsi ketika kami memiliki banyak fitur yang sedikit berguna.

Ketika kita menambahkan regularisasi, fungsi biaya yang baru adalah
$$ $$
$$J(\theta) = \dfrac{1}{m} \sum^{m}_{i=1}\left[-y^{(i)} \; log(h_{\theta}(x^{(i)}))-(1-y^{(i)}) \; log(1-h_{\theta}(x^{(i)}))\right]+\dfrac{\lambda}{2m}\sum^{n}_{j=1}\theta^{2} _{j}$$
$$ $$

Untuk menghitung probabilitas dengan mudah, saya mendefinisikan fungsi prediksi.

In [ ]:
# Menghitung Penurunan Gradien
def gradientdescent(X, y, lmd, alpha, num_iter, print_cost):

    # Memilih initial values Zero
    theta = np.zeros(X.shape[1])
    
    costs = []  
    
    for i in range(num_iter):
        z = np.dot(X, theta)
        h = sigmoid(z)
        
        # Menambah Formula
        reg = lmd / y.size * theta
        reg[0] = 0
        cost = J(h, y)
        
        gradient = np.dot(X.T, (h - y)) / y.size + reg
        theta = theta - alpha * gradient
    
        if print_cost and i % 100 == 0: 
            print('Number of Iterations: ', i, 'Cost : ', cost, 'Theta: ', theta)
        if i % 100 == 0:
            costs.append(cost)
      
    return theta, costs

# Fungsi Prediksi
def predict(X_test, theta):
    z = np.dot(X_test, theta)
    return sigmoid(z)

Terakhir, saya mendefinisikan fungsi utama saya untuk regresi logistik. Ketika kami memiliki lebih dari dua class, saya menggunakan metode one vs all (one vs rest). Secara matematis, dapat ditunjukkan sebagai
$$ $$
$$h_{\theta}^{(i)}(x)=P(y=i \; | \; x;\theta) \;\;\;\;\; (i=1,2,...,n)$$
$$ $$
di mana $n$ adalah jumlah class. Setelah menghitung persamaan di atas, saya memilih class $i$ yang memaksimalkan $h_{\theta}^{(i)}(x)$ untuk menentukan kelas.<span id="8"></span>

In [ ]:
# Fungsi Logistik
def logistic(X_train, y_train, X_test, lmd=0, alpha=0.1, num_iter=30000, print_cost = False):
    # Menambah intercept
    intercept = np.ones((X_train.shape[0], 1))
    X_train = np.concatenate((intercept, X_train), axis=1)
    
    intercept = np.ones((X_test.shape[0], 1))
    X_test = np.concatenate((intercept, X_test), axis=1)

    # 1 melawan sisanya
    u=set(y_train)
    t=[]
    allCosts=[]   
    for c in u:
        # Mengubah label menjadi 0 dan 1
        ynew = np.array(y_train == c, dtype = int)
        theta_onevsrest, costs_onevsrest = gradientdescent(X_train, ynew, lmd, alpha, num_iter, print_cost)
        t.append(theta_onevsrest)
        
        # Menyimpan costs
        allCosts.append(costs_onevsrest)
        
    # Menghitung probabiltas data test
    pred_test = np.zeros((len(u),len(X_test)))
    for i in range(len(u)):
        pred_test[i,:] = predict(X_test,t[i])
    
    # Memilih max probability data test
    prediction_test = np.argmax(pred_test, axis=0)
    
    # Menghitung probabilties data train
    pred_train = np.zeros((len(u),len(X_train)))
    for i in range(len(u)):
        pred_train[i,:] = predict(X_train,t[i])
    
    # Memilih probability data train
    prediction_train = np.argmax(pred_train, axis=0)
    
    d = {"costs": allCosts,
         "Y_prediction_test": prediction_test, 
         "Y_prediction_train" : prediction_train, 
         "learning_rate" : alpha,
         "num_iterations": num_iter,
         "lambda": lmd}
        
    return d

## **3. Split Data**

In [ ]:
all_X = df[['sepal_length','sepal_width','petal_length','petal_width']]
all_y = df['class']

df = df[['sepal_length','sepal_width','petal_length','petal_width','class']]

# Shuffle dataset
shuffle_df = df.sample(frac=1)

# Menentukan pembagian menjadi 80% Train, 20% Test
train_size = int(0.8 * len(df))

# Split dataset 
train_data = shuffle_df[:train_size]
test_data = shuffle_df[train_size:]

X_train = train_data[['sepal_length','sepal_width','petal_length','petal_width']]
y_train = train_data['class']
X_test = test_data[['sepal_length','sepal_width','petal_length','petal_width']]
y_test = test_data['class']

def transform(i):
    if i == 0:
        return 'setosa'
    if i == 1:
        return 'versicolor'
    if i == 2:
        return 'virginica'

## **4. k-Fold Cross Validation**

k-Fold Cross Validation adalah teknik yang sangat berguna untuk memeriksa seberapa baik kinerja model ketika kita menerapkannya pada data independen. Ini sering digunakan untuk menandai masalah yang disebabkan oleh overfitting dan bias seleksi. Namun, ini membawa beban dan waktu pemrosesan data tambahan.

Gambar di bawah menggambarkan validasi silang k-fold. Singkatnya, secara acak membagi data ke k-lipatan, mengambil salah satu lipatan sebagai set pengujian di setiap langkah dan menghitung akurasi.

<img src="https://i.imgur.com/hq45Jfq.png" title="sumber: imgur.com" />

In [ ]:
def cross_validation_split(dataset, folds):
        dataset_split = []
        df_copy = dataset
        fold_size = int(df_copy.shape[0] / folds)
        
        # Loop menyimpan fold
        for i in range(folds):
            fold = []
            # Ketika loop menambah elemen ke folds
            while len(fold) < fold_size:
                # Memilih random element
                r = randrange(df_copy.shape[0])
                # Menentukan indeks untuk element
                index = df_copy.index[r]
                # Menyimpan line
                fold.append(df_copy.loc[index].values.tolist())
                df_copy = df_copy.drop(index)
            #  Menyimpan fold
            dataset_split.append(np.asarray(fold))
            
        return dataset_split

def kfoldCV(dataset, f=5, k=5, model="logistic"):
    data=cross_validation_split(dataset,f)
    result=[]
    # determine training and test sets 
    for i in range(f):
        r = list(range(f))
        r.pop(i)
        for j in r :
            if j == r[0]:
                cv = data[j]
            else:    
                cv=np.concatenate((cv,data[j]), axis=0)
        
        if model == "logistic":       
            c = logistic(cv[:,0:4],cv[:,4],data[i][:,0:4])
            test = c['Y_prediction_test']
            
        # Menghitung accuracy    
        acc=(test == data[i][:,4]).sum()
        result.append(acc/len(test))
        
    return result

## **5. Menghitung Akurasi**

In [ ]:
from statistics import mean 

print("5-Fold Cross Validation")
print("Each fold Size:",int(df.shape[0] / 5))

seed(1)
acc=kfoldCV(df,5)

print("Accuracy:", acc)
print("Average of the Accuracy:", round(mean(acc),2))

5-Fold Cross Validation
Each fold Size: 30
Accuracy: [0.9666666666666667, 0.9, 1.0, 1.0, 0.9666666666666667]
Average of the Accuracy: 0.97
